Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'
```

Imports
=======

``` ipython
import sys
sys.path.insert(0, '../../')

import torch
import gc
from time import perf_counter

from src.network import Network
from src.plasticity import Plasticity
from src.decode import decode_bump
from src.utils import clear_cache

REPO_ROOT = '/home/leon/models/NeuroFlame/'
```

Helpers
=======

``` ipython
def convert_seconds(seconds):
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return h, m, s
```

Testing STP implementation
==========================

From the Plasticity class in src/plasticity.py
----------------------------------------------

First, I we will plot the dynamics of the stp variables for a neuron
with a given rate and Gaussian temporal noise.

``` ipython
tau_list = torch.tensor([0.5, 0.25])
N_BATCH = 2

stp = Plasticity(USE=0.03, TAU_FAC=tau_list, TAU_REC=0.2, DT=0.01, size=(2, 10), STP_TYPE='hansel')

N_STEPS = 500

A_ux = []
u_stp = []
x_stp = []

for _ in range(N_STEPS):
    rate = 10 + 10 * torch.randn((2, 10), device='cuda')

    A_ux.append(stp(rate))
    u_stp.append(stp.u_stp)
    x_stp.append(stp.x_stp)

A_ux = torch.stack(A_ux, dim=1).cpu().numpy()
u_stp = torch.stack(u_stp, dim=1).cpu().numpy()
x_stp = torch.stack(x_stp, dim=1).cpu().numpy()
print(A_ux.shape, u_stp.shape, x_stp.shape)
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=(3*width, height))

ax[0].plot(A_ux[0, :], 'k', label='Aux', alpha=0.25)
ax[1].plot(u_stp[0, :], 'r', label='u', alpha=0.25)
ax[2].plot(x_stp[0, :], 'b', label='x', alpha=0.25)

ax[0].set_xlabel('Step')
ax[0].set_ylabel('$A_{ux}$')

ax[1].set_xlabel('Step')
ax[1].set_ylabel('$u$')

ax[2].set_xlabel('Step')
ax[2].set_ylabel('$x$')

plt.show()
```

Then I will look at the evolution of the steady state value of
A<sub>ux</sub> with the rate for different values of tau<sub>fac</sub>

``` ipython
tau_list = torch.tensor([1.0, .75, .5, .25])
N_BATCH = tau_list.shape[0]

stp = Plasticity(USE=0.03, TAU_FAC=tau_list, TAU_REC=0.2, DT=0.01, size=(N_BATCH, 10), STP_TYPE='hansel')
```

``` ipython
A_ux = []
x = []
u = []

N_RATES = 100
N_STEPS = 300

for i in range(N_RATES): # firing rate
    rates = i + np.sqrt(i) * torch.randn((N_BATCH, N_STEPS, 10), device='cuda')
    for j in range(N_STEPS): # steps before convergence
        A_u_x = stp(rates[:, j])
    A_ux.append(A_u_x)
    u.append(stp.u_stp)
    x.append(stp.x_stp)

A_ux = torch.stack(A_ux, dim=1).cpu().numpy()
u = torch.stack(u, dim=1).cpu().numpy()
x = torch.stack(x, dim=1).cpu().numpy()
print(A_ux.shape, u.shape, x.shape)

```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[3*width, height])

ax[0].plot(A_ux.mean(-1).T, label=tau_list.cpu().numpy())
ax[0].set_xlabel('Rate (Hz)')
ax[0].set_ylabel('$A_{ux}$')

ax[1].plot(u.mean(-1).T, label=tau_list.cpu().numpy())
ax[1].set_xlabel('Rate (Hz)')
ax[1].set_ylabel('$u$')
ax[1].set_ylim([0, 1])

ax[2].plot(x.mean(-1).T, label=tau_list.cpu().numpy())
ax[2].set_xlabel('Rate (Hz)')
ax[2].set_ylabel('$x$')
ax[2].set_ylim([0, 1])
ax[2].legend(frameon=False, loc="right", fontsize=14, title='$\\tau_{fac}$')
plt.show()
```

This is what we expect!

From the Network class in src/network.py
----------------------------------------

### STP shape in the model

``` ipython
model = Network('config_single.yml', REPO_ROOT, IF_STP=1, DT=0.001, DURATION=30, VERBOSE=0, N_NEURON=10)
model.Wab_T = torch.zeros((model.N_NEURON, model.N_NEURON), dtype=torch.float32, device='cuda')
```

``` ipython
Je0_list = np.linspace(0, 100, 50)

ff_inputs = []
for i in Je0_list:
    model.Ja0[:, 0] = i  # here we set the ff input to E to value i in 0 .. 10
    model.VAR_FF[:, 0] = np.sqrt(i)
    ff_inputs.append(model.init_ff_input())

ff_inputs = torch.vstack(ff_inputs)  
```

``` ipython
rates = model(ff_input=ff_inputs, RET_STP=1).cpu().detach().numpy()
print(rates.shape)
```

``` ipython
u_list = model.u_list.cpu().numpy()
x_list = model.x_list.cpu().numpy()
Aux = u_list * x_list
print(u_list.shape, x_list.shape)
```

``` ipython
plt.plot(Aux[:, -1].mean(-1))
plt.xlabel('Rates')
plt.ylabel('Aux')
plt.show()
```

``` ipython
```

### Two neurons

``` ipython
model = Network('config_single.yml', REPO_ROOT, IF_STP=1, DT=0.001, DURATION=10, VERBOSE=0, N_NEURON=2)
model.Wab_T = torch.zeros((model.N_NEURON, model.N_NEURON), dtype=torch.float32, device='cuda')
model.Wab_T[0, 1] = 1.0
```

``` ipython
model.Ja0 = model.Ja0.repeat(1, 1, 2)
model.VAR_FF = model.VAR_FF.repeat(1, 1, 2)
```

``` ipython
Je0_list = np.linspace(0, 100, 50)

model.Ja0[:, 0, 1] = 0  # here we set the ff input to E to value i in 0 .. 10
model.VAR_FF[:, 0, 1] = 0

ff_inputs = []
for i in Je0_list:
    model.Ja0[:, 0, 0] = i  # here we set the ff input to E to value i in 0 .. 10
    model.VAR_FF[:, 0, 0] = np.sqrt(i)

    ff_inputs.append(model.init_ff_input())

ff_inputs = torch.vstack(ff_inputs)
print(ff_inputs.shape)
```

``` ipython
rates = model(ff_input=ff_inputs, RET_STP=1).cpu().detach().numpy()
print(rates.shape)
```

``` ipython
plt.plot(rates[:,-1])
plt.xlabel('FF Input')
plt.ylabel('Rates')
plt.show()
```

``` ipython
u_list = model.u_list.cpu().numpy()
x_list = model.x_list.cpu().numpy()
Aux = u_list * x_list
print(u_list.shape, x_list.shape)
```

``` ipython
plt.plot(rates[:, -1, 0], Aux[:, -1], 'o')
plt.xlabel('Rates')
plt.ylabel('Aux')
plt.show()
```

``` ipython
print(model.J_STP)
print(model.W_stp_T)
```

### Single Trial

``` ipython
model = Network('config_EI.yml', REPO_ROOT, IF_STP=1, DT=0.001, VERBOSE=0)
rates = model(RET_STP=1).cpu().detach().numpy()
```

``` ipython
u_list = model.u_list.cpu().numpy()
x_list = model.x_list.cpu().numpy()
Aux = u_list * x_list
print(u_list.shape, x_list.shape)
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[3*width, height])

ax[0].plot(rates.mean(-1).T, 'k')
ax[0].plot(rates[0, :, :10], alpha=.25)
ax[0].set_xlabel('$Step$')
ax[0].set_ylabel('Rates')

ax[1].plot(u_list.mean(-1).T, 'k')
ax[1].plot(u_list[0, :, :10], alpha=.25)
ax[1].set_xlabel('Step')
ax[1].set_ylabel('$u$')

ax[2].plot(x_list.mean(-1).T, 'k')
ax[2].plot(x_list[0, :, :10], alpha=.25)
ax[2].set_xlabel('Step')
ax[2].set_ylabel('$x$')

plt.show()
```

``` ipython
m0, m1, phi = decode_bump(rates, axis=-1)
print(phi.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].imshow(rates.T, aspect='auto', cmap='jet', vmin=0, vmax=10, origin='lower')
ax[0].set_ylabel('Pref. Location (°)')
ax[0].set_yticks(np.linspace(0, model.Na[0].cpu(), 5), np.linspace(0, 360, 5).astype(int))
ax[0].set_xlabel('Step')

ax[1].plot(phi[0] * 180 / np.pi)
ax[1].set_yticks(np.linspace(0, 360, 5).astype(int), np.linspace(0, 360, 5).astype(int))
ax[1].set_ylabel('Pref. Location (°)')
ax[1].set_xlabel('Step')
plt.show()
```

``` ipython
#  plt.plot(rates[-1], Aux[-1], 'o')
```

``` ipython
print(torch.cuda.memory_allocated()/100000)
del model
clear_cache()
print(torch.cuda.memory_allocated()/100000)
```

``` ipython
```

### Rates vs Ie

``` ipython
model = Network('config_2pop.yml', REPO_ROOT, IF_STP=1, DT=0.001, GAIN=0.5, VERBOSE=0)
```

``` ipython
Je0_list = np.linspace(0, 10, 10)
print(Je0_list)

ff_inputs = []
for i in Je0_list:
    model.Ja0[:, 0] = i  # here we set the ff input to E to value i in 0 .. 10      
    ff_inputs.append(model.init_ff_input())

ff_inputs = torch.vstack(ff_inputs)  
```

``` ipython
rates_Je0 = model(ff_inputs, RET_STP=1).cpu().detach().numpy()
```

``` ipython
u_list = model.u_list.cpu().numpy()
x_list = model.x_list.cpu().numpy()
print(u_list.shape, x_list.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

mean_rates = rates_Je0[:,-1].mean(-1)

ax[0].plot(Je0_list, mean_rates)
ax[0].set_xlabel('$J_{E0}$')
ax[0].set_ylabel('$<Rates>_i$')
# ax[0].set_ylim([0, 60])

ax[1].plot(rates_Je0.mean(-1).T)
ax[1].set_xlabel('Step')
ax[1].set_ylabel('Rates')
# ax[1].set_ylim([0, 60])
plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

mean_rates = u_list[:,-1].mean(-1) * x_list[:,-1].mean(-1)

ax[0].plot(Je0_list, mean_rates)
ax[0].set_xlabel('$J_{E0}$')
ax[0].set_ylabel('$A_{ux}$')

ax[1].plot(u_list.mean(-1).T * x_list.mean(-1).T)
ax[1].set_xlabel('Step')
ax[1].set_ylabel('$A_{ux}$')

plt.show()

```

``` ipython
print(torch.cuda.memory_allocated()/100000)
del model
clear_cache()
print(torch.cuda.memory_allocated()/100000)
```

### Rates vs Jee

``` ipython
model = Network('config_2pop.yml', REPO_ROOT, IF_STP=1, DT=0.001, GAIN=0.5, VERBOSE=0)
```

``` ipython
model.IF_BATCH_J = 1

Jee_list = torch.linspace(0.5, 3, 10, device='cuda')
model.Jab_batch = Jee_list.unsqueeze(-1) * model.Jab[0, 0]

model.IF_STP = 1
model.N_BATCH = model.Jab_batch.shape[0]
model.VERBOSE = 0

rates_Jee = model(RET_STP=1).cpu().detach().numpy()
```

``` ipython
u_list = model.u_list.cpu().numpy()
x_list = model.x_list.cpu().numpy()
print(u_list.shape, x_list.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

mean_rates = rates_Jee[:,-1].mean(-1)

ax[0].plot(Jee_list.cpu().numpy(), mean_rates)
ax[0].set_xlabel('$J_{EE}$')
ax[0].set_ylabel('$<Rates>_i$')
# ax[0].set_ylim([0, 60])

ax[1].plot(rates_Jee.mean(-1).T)
ax[1].set_xlabel('$J_{EE}$')
ax[1].set_ylabel('Rates')
# ax[1].set_ylim([0, 60])
plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

mean_rates = u_list[:,-1].mean(-1) * x_list[:,-1].mean(-1)

ax[0].plot(Jee_list.cpu(), mean_rates)
ax[0].set_xlabel('$J_{EE}$')
ax[0].set_ylabel('$A_{ux}$')

ax[1].plot(u_list.mean(-1).T * x_list.mean(-1).T)
ax[1].set_xlabel('Step')
ax[1].set_ylabel('$A_{ux}$')

plt.show()

```

``` ipython
print(torch.cuda.memory_allocated()/100000)
del model
clear_cache()
print(torch.cuda.memory_allocated()/100000)
```

### Rates vs Use

``` ipython
model = Network('config_2pop.yml', REPO_ROOT, IF_STP=1, DT=0.001, GAIN=0.5, VERBOSE=0)
```

``` ipython
model.USE = torch.linspace(0.01, 0.1, 10, device='cuda')
model.N_BATCH = model.USE.shape[0]

rates_use = model(RET_STP=1).cpu().detach().numpy()
print(rates_use.shape)
```

``` ipython
u_list = model.u_list.cpu().numpy()
x_list = model.x_list.cpu().numpy()
print(u_list.shape, x_list.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

mean_rates = rates_use[:,-1].mean(-1)

ax[0].plot(model.USE.cpu().numpy(), mean_rates)
ax[0].set_xlabel('Use')
ax[0].set_ylabel('$<Rates>_i$')
# ax[0].set_ylim([0, 60])

ax[1].plot(rates_use.mean(-1).T)
ax[1].set_xlabel('Step')
ax[1].set_ylabel('Rates')
# ax[1].set_ylim([0, 60])
plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

Aux = u_list * x_list

ax[0].plot(model.USE.cpu(), Aux[:, -1].mean(-1))
ax[0].set_xlabel('$Use$')
ax[0].set_ylabel('$A_{ux}$')

ax[1].plot(Aux.mean(-1).T)
ax[1].set_xlabel('Step')
ax[1].set_ylabel('$A_{ux}$')

plt.show()
```

``` ipython
print(torch.cuda.memory_allocated()/100000)
del model
clear_cache()
print(torch.cuda.memory_allocated()/100000)
```

``` ipython
```

### Rates vs Tau fac

``` ipython
model = Network('config_2pop.yml', REPO_ROOT, IF_STP=1, DT=0.001, GAIN=0.5, VERBOSE=0, DURATION=10)
```

``` ipython
model.TAU_FAC = torch.linspace(0.25, 1.0, 10, device='cuda')
model.N_BATCH = model.TAU_FAC.shape[0]

rates_fac = model(RET_STP=1).cpu().detach().numpy()
print(rates_fac.shape)
```

``` ipython
u_list = model.u_list.cpu().numpy()
x_list = model.x_list.cpu().numpy()
print(u_list.shape, x_list.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

mean_rates = rates_fac[:,-1].mean(-1)

ax[0].plot(model.TAU_FAC.cpu().numpy()*1000, mean_rates, 'k')
ax[0].plot(model.TAU_FAC.cpu().numpy()*1000, rates_fac[:, -1, :10], alpha=.25)
ax[0].set_xlabel('$\\tau_{fac}$')
ax[0].set_ylabel('$<Rates>_i$')

ax[1].plot(rates_fac.mean(-1).T)
ax[1].set_xlabel('Step')
ax[1].set_ylabel('Rates')
plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

mean_Aux = u_list[:,-1].mean(-1) * x_list[:,-1].mean(-1)

ax[0].plot(model.TAU_FAC.cpu().numpy() * 1000, mean_Aux)
ax[0].set_xlabel('$\\tau_{fac}$')
ax[0].set_ylabel('$A_{ux}$')

ax[1].plot(u_list.mean(-1).T * x_list.mean(-1).T)
ax[1].set_xlabel('Step')
ax[1].set_ylabel('$A_{ux}$')

plt.show()
```

``` ipython
print(torch.cuda.memory_allocated()/100000)
del model
clear_cache()
print(torch.cuda.memory_allocated()/100000)
```

``` ipython
```